In [1]:
from keras.models import Sequential
from keras.layers.core import Activation, Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical

import clamv

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 26 days
Using Theano backend.
ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.
/home/dcs/csunbd/anaconda3/lib/python3.5/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [2]:
X, y = clamv.get_images()

In [3]:
sample, h, w = clamv.get_image_shape()
classes = clamv.get_num_classes()

In [4]:
from sklearn.preprocessing import normalize

In [5]:
X = normalize(X)

In [15]:
# Reshape into a 4D tensor
X = X.reshape(X.shape[0], 1, h, w)

In [31]:
from sklearn.base import BaseEstimator, ClassifierMixin
class CNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, classes, img_h, img_w, epochs):
        self.classes = classes
        self.epochs = epochs
        
        self.img_h = img_h
        self.img_w = img_w

    def fit(self, X, y):
        self.cnn = Sequential()
        self.cnn.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(1, self.img_h, self.img_w)))
        self.cnn.add(Activation('relu'))
        self.cnn.add(Convolution2D(32, 3, 3, border_mode='valid'))
        self.cnn.add(Activation('relu'))
        self.cnn.add(Dropout(0.4))
        self.cnn.add(MaxPooling2D(pool_size=(2,2)))
        self.cnn.add(Convolution2D(32, 3, 3, border_mode='valid'))
        self.cnn.add(Activation('relu'))
        self.cnn.add(Convolution2D(32, 3, 3, border_mode='valid'))
        self.cnn.add(Activation('relu'))
        self.cnn.add(Dropout(0.4))

        self.cnn.add(Flatten())
        self.cnn.add(Dense(128))
        self.cnn.add(Activation('relu'))
        self.cnn.add(Dropout(0.5))
        self.cnn.add(Dense(self.classes))
        self.cnn.add(Activation('softmax'))
        
        y_categorical = to_categorical(y, self.classes)
        self.cnn.compile(loss="categorical_crossentropy", optimizer='adam')
        
        self.cnn.fit(X, y_categorical,  batch_size=128, nb_epoch=self.epochs, show_accuracy=True)
        return self
    
    def predict(self, X):
        return self.cnn.predict_classes(X)

In [32]:
model = CNNClassifier(classes, h, w, 50)

In [33]:
clamv.test(model, X, y, "results/cnn")

KeyboardInterrupt: 